### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [3]:
# Count the number of unique SNs
num_players = purchase_data["SN"].nunique()

# Include the total number of players in a DataFrame to be displayed
total_players = pd.DataFrame(data = {"Total Players" : num_players}, index = [0])
total_players

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
# First group the data by Item ID
grouped_data = purchase_data.groupby("Item ID")

# Count the number of unique Item IDs. This is the lenghth of the grouped DataFrame
num_items = len(grouped_data)

# Calculate the average price per item (this should just be a series of Item -> Price)
avrg_prices = grouped_data.mean()["Price"]

#Now calculate the average of that series to get the average price of all products. Rounded to 2 decimal places
avrg_price = round(avrg_prices.mean(), 2)


# Count all the purchases. This should be the length of the original dataframe
num_purchases = len(purchase_data)

# Sum all the revenue for the sales per item (This should be a series of Item -> Revenue)
revenue_per_item = grouped_data.sum()["Price"]

# Sum all the values in revenue_per_item for the total revenue
total_revenue = revenue_per_item.sum()

# Put everything on a DataFrame
purchase_analysis = pd.DataFrame({"Number of Unique Items":num_items,
                                 "Average Price":avrg_price,
                                 "Number of Purchases":num_purchases,
                                 "Total Revenue":total_revenue}, index=[0])

# Format Average Price and Revenue to have dollar signs
purchase_analysis["Average Price"] = purchase_analysis["Average Price"].map('${:,.2f}'.format)
purchase_analysis["Total Revenue"] = purchase_analysis["Total Revenue"].map('${:,.2f}'.format)

purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.04,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
# First group the data by Gender
gender_data = purchase_data.groupby(["Gender"])

# Count how many unique players are male/female/other
counts_gender = gender_data["SN"].nunique()
percentage_gender = round(counts_gender*100/num_players,2)

# Put results in a DataFrame
gender_dem = pd.DataFrame({"Count":counts_gender.values,
                           "Percentage of Players":percentage_gender.values}, index = ["Female", "Male", "Other / Non Disclosed"])

#Sort values from largest to smallest
gender_dem = gender_dem.sort_values("Count", ascending = False)

# Format percentages
gender_dem["Percentage of Players"] = gender_dem["Percentage of Players"].map('{:,.2f}%'.format)

gender_dem

,Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [6]:
# Count how many purchases are made per gender
purchase_count = gender_data.count()["SN"]

# Take the average of the item price that is bought per gender
avrg_purchase = gender_data.mean()["Price"]

# Sum all the prices of the items bought per gender
total_purchase = gender_data.sum()["Price"]

#-----------------------------------------------------------------------

# Group the original data by both gender and user
user_purchases = purchase_data.groupby(["SN","Gender"])

# Get the total amount of money spent per user
total_user_purchase = user_purchases.sum()["Price"]

# Group now only by gender to get the average total spent per user, per gender
average_total_gender = total_user_purchase.groupby("Gender").mean()

# Place data inside a DataFrame
gender_purchase_analysis = pd.DataFrame({"Purchase Count":purchase_count.values,
                                         "Average Purchase Price":avrg_purchase.values,
                                         "Total Purchase Value":total_purchase.values,
                                         "Average Total Purchase per Person":average_total_gender.values},
                                        index = ["Female","Male","Other / Non Disclosed"])

# Format to two decimal places and include dollar sign
gender_purchase_analysis["Average Purchase Price"] = gender_purchase_analysis["Average Purchase Price"].map('${:,.2f}'.format)
gender_purchase_analysis["Average Total Purchase per Person"] = gender_purchase_analysis["Average Total Purchase per Person"].map('${:,.2f}'.format)
gender_purchase_analysis["Total Purchase Value"] = gender_purchase_analysis["Total Purchase Value"].map('${:,.2f}'.format)

gender_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [7]:
# Define bins and labels
bins = [0,9,14,19,24,29,34,39,999]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

#Group data by user
users_grouped = purchase_data.groupby("SN")

# Take the mean age per player (so that the data is actually grouped) and put their ages in bins
binned_ages = pd.cut(users_grouped.mean()["Age"], bins, labels = labels)

# Count how many players are in each bin
ages_counts = binned_ages.value_counts()

# Reorder the age counts to be in the same order as the labels
ages_counts = ages_counts.loc[labels]

# Calculate the percentages of each age
ages_percentage = 100*ages_counts/num_players

# Put the result in a DataFrame
age_dem = pd.DataFrame({"Total Count":ages_counts, "Percentage of Players":ages_percentage}, index = labels)

# Format percentages
age_dem["Percentage of Players"] = age_dem["Percentage of Players"].map('{:,.2f}%'.format)

age_dem

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [8]:
# Copy the data into a new array, just in case
purchase_data_age = purchase_data

# Bin the data by age
purchase_data_age["Age Group"] = pd.cut(purchase_data_age["Age"], bins, labels = labels)

# Group the data by age group
grouped_data_age = purchase_data_age.groupby("Age Group")

# Get counts per age group
num_purchases_age = grouped_data_age.count()["SN"]

# Get average purchase price per age group
avrg_purchases_age = grouped_data_age.mean()["Price"]

# Get total purchase price per age group
total_purchases_age = grouped_data_age.sum()["Price"]

#-----------------------------------------------------------------------

# Group the copied data by both age group and user
user_purchases_age = purchase_data.groupby(["SN","Age Group"])

# Get the total amount of money spent per user
total_user_purchase_age = user_purchases_age.sum()["Price"]

# Group now only by age group to get the average total spent per user, per age group
average_total_age = total_user_purchase_age.groupby("Age Group").mean()

# Place data inside DataFrame
age_purchase_analysis = pd.DataFrame({"Purchase Count":num_purchases_age,
                                      "Average Purchase Price":avrg_purchases_age,
                                      "Total Purchase Value":total_purchases_age,
                                      "Average Total Purchase per Person":average_total_age},
                                      index = labels)

# Format to two decimal places and include dollar sign
age_purchase_analysis["Average Purchase Price"] = age_purchase_analysis["Average Purchase Price"].map('${:,.2f}'.format)
age_purchase_analysis["Average Total Purchase per Person"] = age_purchase_analysis["Average Total Purchase per Person"].map('${:,.2f}'.format)
age_purchase_analysis["Total Purchase Value"] = age_purchase_analysis["Total Purchase Value"].map('${:,.2f}'.format)

age_purchase_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [9]:
# Group the data by user
gropued_data_sn = purchase_data.groupby("SN")

# Count how many purchases each user has made
num_purchases_sn = gropued_data_sn.count()["Purchase ID"]

# Get the average money each user has spent
avrg_purchases_sn = gropued_data_sn.mean()["Price"]

# Get the total money each user has spent
total_purchases_sn = gropued_data_sn.sum()["Price"]

# Place data inside DataFrame
sn_data = pd.DataFrame({"Purchase Count":num_purchases_sn,
                        "Average Purchase Price":avrg_purchases_sn,
                        "Total Purchase Value":total_purchases_sn},
                       index = total_purchases_sn.index)

# Sort based on who spent the most money
sn_data = sn_data.sort_values(by = "Total Purchase Value",ascending=False)

# Format to two decimal places and include dollar sign
sn_data["Average Purchase Price"] = sn_data["Average Purchase Price"].map('${:,.2f}'.format)
sn_data["Total Purchase Value"] = sn_data["Total Purchase Value"].map('${:,.2f}'.format)

sn_data.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
# Group the data by product id and name to keep both pieces of information
gropued_data_product = purchase_data.groupby(["Item ID", "Item Name"])

# Count how many times the product was bought
num_purchases_product = gropued_data_product.count()["Purchase ID"]

# Get the item price
price_product = gropued_data_product.mean()["Price"]

# Get the total money each product has made
total_purchases_product = gropued_data_product.sum()["Price"]

# Place data inside DataFrame
product_data = pd.DataFrame({"Purchase Count":num_purchases_product,
                             "Item Price":price_product,
                             "Total Purchase Value":total_purchases_product},
                             index = total_purchases_product.index)

# Sort based on which product was bought the most times
product_data_popular = product_data.sort_values(by = "Purchase Count",ascending=False)

# Format to two decimal places and include dollar sign
product_data_popular["Item Price"] = product_data_popular["Item Price"].map('${:,.2f}'.format)
product_data_popular["Total Purchase Value"] = product_data_popular["Total Purchase Value"].map('${:,.2f}'.format)

product_data_popular.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [12]:
# Sort based on which product made the most money
product_data_profit = product_data.sort_values(by = "Total Purchase Value",ascending=False)

# Format to two decimal places and include dollar sign
product_data_profit["Item Price"] = product_data_profit["Item Price"].map('${:,.2f}'.format)
product_data_profit["Total Purchase Value"] = product_data_profit["Total Purchase Value"].map('${:,.2f}'.format)

product_data_profit.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80


# Trends

From the analysis done above, we can see that the game is highly male-dominated, with male players making up 84.03% of all the players. However, looking at the purchases per gender, we can see that male players have the lowest average spending. This might indicate that marketing the game to non-male users might significantly increase profits.

Regarding the age of the players, almost half of the users are between the ages of 20 - 24. Furthermore, 76.74% of all players are between the ages of 15-29. Although the age gruop with the highest average spending per user is 35-39, they make up for less than 6% of all players.

As for the purchases of the players themselves, it is interesting to note that most of them don't make more than a couple of purchases. The most popular and profitable item has been "Oathbreaker, Last Hope of the Breaking Storm", which was only bought 12 times. Maybe the game could profit from a mechanic where repeated purchases are favored (like selling bundles of a virtual currency, or something like health posions / extra lives)